# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [8]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
#     llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
#     llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-02-03 19:39:50,209 - 139724257950592 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Kicking off the Crew

In [9]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: I need to gather the most up-to-date information regarding inflation in the US and its potential impact on the stock market in 2024. This will involve searching for recent news articles and analysis related to this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 news updates\"}"
## T



# Agent: Lead Market Analyst
## Thought: Thought: I need to read more relevant articles to gather comprehensive insights on US inflation and its impact on the stock market for 2024.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.apnews.com/article/stocks-markets-inflation-korea-25b0400f641fc1b0fc6036797d63726f\"}"
## Tool Output: 
Stock market today: Wall Street rips higher after inflation data and strong bank profits | AP News
Menu
Menu
World
SECTIONS
Mideast Wars
Russia-Ukraine War
Español
China
Asia Pacific
Latin America
Europe
Africa
TOP STORIES
In Kentucky bourbon country, the newest trade war feels like a hangover that won't go away
What is USAID? Explaining the US foreign aid agency and why Trump and Musk want to end it
Tariff threats take aim at fentanyl trafficking. Here's how the drug reaches the US
Newsletters
The Morning Wire
Our flagship newsletter breaks down the biggest headlines of the day.
The Afternoon Wire
Get caught up on what



# Agent: Lead Market Analyst
## Final Answer: 
---

**Summary Report on US Inflation and Its Impact on the Stock Market in 2024**

**1. Overview of US Inflation Trends:**
   - Recent data indicates a decline in US inflation rates with the annual inflation rate recorded at 2.2% in August 2024, the lowest since February 2021. This decline has fostered expectations for aggressive interest rate cuts by the Federal Reserve in the coming months, potentially by up to 0.5 percentage points.
   - As inflation rates decrease, the S&P 500 index has soared to record highs, registering a 34% year-on-year gain, largely attributed to favorable inflation reports and economic policies in response to inflationary pressures.

**2. Stock Market Performance:**
   - The positive inflation data has resulted in significant gains across major stock indices. The S&P 500 jumped 1.8%, the Dow Jones Industrial Average increased by 703 points (1.7%), and the Nasdaq Composite gained 2.5%. This surge is coupled wit

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.14it/s]




# Agent: Chief Data Strategist
## Thought: Thought: I have gathered more recent search results regarding US inflation and its impact on the stock market for 2024. I will now extract relevant information from these sources to synthesize actionable insights for the comprehensive analysis report.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation trends 2024\", \"website\": \"https://www.forrester.com\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.3% t

 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'russellinvestments.com': No scheme supplied. Perhaps you meant https://russellinvestments.com?.
 Tool Search in a specific website accepts these inputs: Search in a specific website(search_query: 'string', website: 'string') - A tool that can be used to semantic search a query from a specific URL content. search_query: 'Mandatory search query you want to use to search a specific website', website: 'Mandatory valid website URL you want to search on'



# Agent: Creative Content Director
## Thought: Thought: I need to extract relevant insights from the gathered content to create high-quality blog posts and social media updates related to US inflation and its impact on the stock market in 2024.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 insights\", \"website\": \"russellinvestments.com\"}"
## Tool Output: 

I encountered a

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]




# Agent: Creative Content Director
## Thought: Thought: I will search for insights regarding US inflation and its impact on the stock market in 2024 from a different source to gather necessary data for creating engaging content pieces.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024 insights\", \"website\": \"https://www.blackrock.com\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.3% this year, up from the 2.2% increas



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Understanding US Inflation and Its Impacts on the Stock Market in 2024

## Introduction
As we navigate through 2024, the decline in US inflation rates has triggered significant shifts in the stock market, presenting both opportunities and challenges for investors. In this comprehensive analysis, we will explore the latest trends in US inflation, the performance of major stock indices, and what lies ahead for investors in the next fiscal year.

---

## 1. Current Inflation Trends
According to recent reports, the annual inflation rate in the US reached a low of **2.2%** in August 2024—the lowest since February 2021. This significant decline has raised expectations among analysts for aggressive interest rate cuts by the Federal Reserve, potentially by up to **0.5%**. Overall, inflation is projected to average around **2.6%** throughout 2024, down from **4.1%** in 2023.

![Inflation Trend Chart](http://example.com/inflation_t

## Social Content

In [10]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🚨 **BREAKING**: US inflation hits **2.2%** - the
lowest since Feb 2021! 📉 What does this mean for
the stock market? Analysts predict potential
interest rate cuts. Check our latest insights!
#Inflation2024 #StockMarketInsights
--------------------------------------------------
Instagram
📊💡 **Inflation is declining in the US**, leading
to substantial gains in major stock indices! As of
Sept 2024, the S&P 500 has surged with a **34%**
year-on-year increase! Swipe to discover what the
future holds! #Finance #Investing #MarketTrends
--------------------------------------------------
LinkedIn
Join us as we analyze the recent decline in US
inflation and its impact on the stock market for
fiscal year 2024. With expectations of interest
rate cuts and improving investor sentiment,
discover how to navigate the forthcoming
opportunities and challenges. #Finance
#InvestmentStrategy
--------------------------------------------------


## Blog Post

In [11]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding US Inflation and Its Impacts on the Stock Market in 2024

## Introduction
As we navigate through 2024, the decline in US inflation rates has triggered significant shifts in the stock market, presenting both opportunities and challenges for investors. In this comprehensive analysis, we will explore the latest trends in US inflation, the performance of major stock indices, and what lies ahead for investors in the next fiscal year.

---

## 1. Current Inflation Trends
According to recent reports, the annual inflation rate in the US reached a low of **2.2%** in August 2024—the lowest since February 2021. This significant decline has raised expectations among analysts for aggressive interest rate cuts by the Federal Reserve, potentially by up to **0.5%**. Overall, inflation is projected to average around **2.6%** throughout 2024, down from **4.1%** in 2023.

![Inflation Trend Chart](http://example.com/inflation_trend_chart)

As inflation eases, many experts are cautiously optimistic about the potential rebound in consumer spending and business investments. A stable inflation environment often fosters consumer confidence, which can lead to increased economic activity. Therefore, understanding these trends will be crucial for investors as they structure their portfolios in response to shifting economic conditions.

---

## 2. Stock Market Performance
The positive inflation reports have translated into substantial gains across major stock indices. As of the end of September 2024:
- The **S&P 500** has surged with a remarkable **34% year-on-year gain**.
- The **Dow Jones Industrial Average** increased by **703 points (1.7%)**.
- The **Nasdaq Composite** rose by **2.5%** in tandem with strong profit reports from major banks.

This momentum is also enhanced by expectations of increased economic activity due to potential interest rate cuts. The stock market's reaction underscores a broader investor sentiment that favors equities, particularly in sectors poised to benefit from lower borrowing costs and increased consumer resilience.

---

## 3. Investor Sentiment and Economic Outlook
While the outlook appears optimistic, volatility remains likely as investors respond to evolving narratives about inflation and Federal Reserve policies. As noted by analysts, focusing on economic indicators is critical, as they may signal whether inflationary pressures will persist or subside. 

Experts also caution about ongoing geopolitical risks that could impact growth rates and investor confidence. Elements such as trade negotiations and developments in global supply chains can complicate the macroeconomic landscape, requiring investors to stay agile and informed.

---

## 4. Global Implications
The decline in US inflation is setting a positive tone globally, with European indices like the **FTSE 100** and **DAX** experiencing gains. The **International Monetary Fund (IMF)** has indicated that a synchronized reduction in inflation could foster global economic recovery, alleviating pressure in other markets.

This interconnectedness emphasizes the importance for investors to adopt a global perspective in their strategies. As inflation rates decrease in the US, the ripple effect can foster recovery in emerging and developed markets alike, presenting potential investment opportunities internationally.

---

## 5. Future Projections
Looking ahead, economists predict that if inflation stabilizes or continues its downward trajectory, we might see further interest rate cuts in 2024, potentially supporting continued growth in equity markets. However, continued monitoring of inflation trends and labor market data will be essential in gauging stock market movements and investment strategies.

Investors should consider diversifying their portfolios to balance risks associated with potential market fluctuations while focusing on sectors that typically flourish in lower interest environments. Staying informed and ready to adjust tactics in response to economic changes will be vital for success in the investment landscape of 2024.

---

## Conclusion
As inflation concerns ease and the Federal Reserve is likely to cut interest rates, we may be on the brink of a new phase in the stock market. While enthusiasm is justified, investors should prepare for the inevitable unpredictability of market dynamics in 2024. An informed strategy, based on a comprehensive understanding of economic indicators, will be paramount in navigating this evolving landscape.

---